In [ ]:
# This script reads a csv file containing run length encoded segmentation labels,
# creates the encoded binary images and saves them 

# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Path to csv file containing run length encoded labels
csv_path ='/content/drive/MyDrive/Deep_learning/project/data/train.csv'

# Path to train images
train_path ='/content/drive/MyDrive/Deep_learning/project/data/train/'

# Path to save label images
label_path ='/content/drive/MyDrive/Deep_learning/project/data/labels/'

Mounted at /content/drive


In [ ]:
# Import modules
import numpy as np
import matplotlib.image as mpimg
from matplotlib import cm
import pandas as pd


In [ ]:
def rle_decode(mask_rle, shape, color=1):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros((shape[0] * shape[1], shape[2]), dtype=np.float32)
    for lo, hi in zip(starts, ends):
        img[lo : hi] = color
    return img.reshape(shape)


def build_masks(labels, input_shape, colors=True):
    height, width = input_shape
    if colors:
        mask = np.zeros((height, width, 3))
        for label in labels:
            mask += rle_decode(label, shape=(height, width , 3), color=np.random.rand(3))
    else:
        mask = np.zeros((height, width, 1))
        for label in labels:
            mask += rle_decode(label, shape=(height, width, 1))
    mask = mask.clip(0, 1)
    return mask

In [ ]:
train_csv = pd.read_csv(csv_path)

sample_ids = train_csv["id"].unique()

In [ ]:
def save_images(sample_ids):
    '''
    Function to create and save images
    
    sample_ids - list of ids

    '''
    
    for idx, sample_id in enumerate(sample_ids):
        
        sample_rles = train_csv.loc[train_csv['id'] == sample_id]['annotation'].values

        sample_mask = build_masks(sample_rles, (520, 704), colors=False)

        sample_mask = (np.squeeze(sample_mask, axis=2)) 

        mpimg.imsave(label_path + sample_id + '.png', sample_mask, cmap = cm.binary) 




In [ ]:
save_images(sample_ids)